# Hoja de Trabajo \# 4

---


por Josué Obregón <br>
DS6011 - Feature Engineering <br>
UVG Masters - Escuela de Negocios<br>


## Objetivos

El objetivo de esta hoja de trabajo  es presentar al estudiante diferentes técnicas de codificación de variables categóricas. Éstas técnicas incluyen codificadores clásicos, de contraste y codificadores supervisados o bayesianos.

También se busca que el estudiante practique la utilización de éstas técnicas con las librerías disponibles en el lenguaje Python.


## Importación de librerías y carga de los datos a varios pandas [DataFrames](https://pandas.pydata.org/pandas-docs/version/1.1.5/reference/frame.html)




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [3]:
!mkdir data

In [4]:
import gdown

urls = ['https://drive.google.com/uc?export=download&id=16AGQw1nM9NYILv2aSZaSNSn9jBPByWPq', # okc_train  https://drive.google.com/file/d/16AGQw1nM9NYILv2aSZaSNSn9jBPByWPq/view?usp=sharing
        ]
outputs = ['okc_train.csv']
for url,output in zip(urls,outputs):
  gdown.download(url, f'data/{output}', quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=16AGQw1nM9NYILv2aSZaSNSn9jBPByWPq
To: /content/data/okc_train.csv
12.3MB [00:00, 135MB/s]


In [5]:
df = pd.read_csv('data/okc_train.csv',index_col=0)

In [6]:
df.head()

,age,diet,drinks,drugs,education,height,income,last_online,offspring,pets,religion,sign,smokes,status,where_state,where_town,religion_modifer,sign_modifer,cpp,cpp_fluently,cpp_okay,cpp_poorly,lisp,lisp_fluently,lisp_okay,lisp_poorly,asian,black,hispanic_latin,indian,middle_eastern,native_american,other,pacific_islander,white,essay_length,Class,profile
1,22,strictly_anything,socially,never,working_on_college_university,75,missing,3,doesnt_have_kids_but_might_want_them,likes_dogs_and_likes_cats,agnosticism,gemini,sometimes,single,california,south_san_francisco,and_very_serious_about_it,sign_mod_missing,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,3.420945,other,1
4,23,vegetarian,socially,drugs_missing,working_on_college_university,71,inc20000,3,doesnt_want_kids,likes_cats,religion_missing,pisces,no,single,california,berkeley,religion_mod_missing,sign_mod_missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.671173,other,3
6,29,mostly_anything,socially,drugs_missing,graduated_from_college_university,67,missing,2,doesnt_have_kids_but_might_want_them,likes_cats,atheism,taurus,no,single,california,san_francisco,religion_mod_missing,sign_mod_missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.391112,stem,5
8,31,mostly_anything,socially,never,graduated_from_college_university,65,missing,2,doesnt_have_kids_but_wants_them,likes_dogs_and_likes_cats,christianity,sagittarius,no,single,california,san_francisco,religion_mod_missing,sign_mod_missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.090611,other,6
10,37,mostly_anything,not_at_all,never,working_on_two_year_college,65,missing,3,kids_missing,likes_dogs_and_likes_cats,atheism,cancer,no,single,california,san_mateo,and_laughing_about_it,but_it_doesnt_matter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.382377,other,7


In [7]:
df['drinks'].value_counts()

socially          27521
rarely             3981
often              3413
not_at_all         2157
drinks_missing     1208
very_often          319
desperately         210
Name: drinks, dtype: int64

In [8]:
df['status'].value_counts()

single            36014
seeing_someone     1354
available          1234
married             204
unknown               3
Name: status, dtype: int64

Pequeño dataset de prueba para algunas explicaciones y para las figuras en las diapositivas


In [9]:
df_test = pd.DataFrame({
... 'City': ['SF', 'SF', 'SF', 'NYC', 'NYC', 'NYC',
... 'Seattle', 'Seattle', 'Seattle'],
... 'Rent': [3999, 4000, 4001, 3499, 3500, 3501, 2499, 2500, 2501]
... })

In [10]:
df_test

,City,Rent
0,SF,3999
1,SF,4000
2,SF,4001
3,NYC,3499
4,NYC,3500
5,NYC,3501
6,Seattle,2499
7,Seattle,2500
8,Seattle,2501


Para esta hoja de trabajo vamos a utilizar la librería [category_encoders](http://contrib.scikit-learn.org/category_encoders/index.html), la cual es compatible con scikit-learn.

In [11]:
!pip install category_encoders

     |████████████████████████████████| 81kB 7.4MB/s 


In [15]:
col = 'drinks'

# Unsupervised Encoders

## Classic Encoders

### One-Hot Encoder

In [17]:
from category_encoders import OneHotEncoder

In [18]:
onehot_enc = OneHotEncoder(use_cat_names=True)

In [19]:
onehot_enc.fit_transform(df[[col]])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



,drinks_socially,drinks_not_at_all,drinks_rarely,drinks_often,drinks_drinks_missing,drinks_very_often,drinks_desperately
1,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0
6,1,0,0,0,0,0,0
8,1,0,0,0,0,0,0
10,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
59940,1,0,0,0,0,0,0
59943,0,0,0,1,0,0,0
59944,0,1,0,0,0,0,0
59945,1,0,0,0,0,0,0


In [20]:
onehot_enc.mapping[0]['mapping']

,drinks_socially,drinks_not_at_all,drinks_rarely,drinks_often,drinks_drinks_missing,drinks_very_often,drinks_desperately
1,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0
5,0,0,0,0,1,0,0
6,0,0,0,0,0,1,0
7,0,0,0,0,0,0,1
-1,0,0,0,0,0,0,0
-2,0,0,0,0,0,0,0


### Binary Encoder

In [21]:
from category_encoders import BinaryEncoder

In [27]:
binary_enc = BinaryEncoder()

In [31]:
binary_enc.fit_transform(df[[col]])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



,drinks_0,drinks_1,drinks_2,drinks_3
1,0,0,0,1
4,0,0,0,1
6,0,0,0,1
8,0,0,0,1
10,0,0,1,0
...,...,...,...,...
59940,0,0,0,1
59943,0,1,0,0
59944,0,0,1,0
59945,0,0,0,1


In [32]:
binary_enc.base_n_encoder.mapping[0]['mapping']

,drinks_0,drinks_1,drinks_2,drinks_3
1,0,0,0,1
2,0,0,1,0
3,0,0,1,1
4,0,1,0,0
5,0,1,0,1
6,0,1,1,0
7,0,1,1,1
-1,0,0,0,0
-2,0,0,0,0


### Frequency or Count Encoder

In [39]:
from category_encoders import CountEncoder

In [40]:
count_enc = CountEncoder(normalize=False)

In [35]:
df[col].value_counts()

socially          27521
rarely             3981
often              3413
not_at_all         2157
drinks_missing     1208
very_often          319
desperately         210
Name: drinks, dtype: int64

Intentemos utilizando las funciones de combinacion de grupos

In [42]:
count_enc = CountEncoder(combine_min_nan_groups=True, min_group_size=500, min_group_name='otros', normalize=True)

### Ordinal Encoder

In [ ]:
from category_encoders import OrdinalEncoder

In [ ]:
ord_enc = OrdinalEncoder()

###Feature Hashing

## Contrast Encoders

### Diferencia entre One-hot Encoding y Dummy Coding

Con la codificación one-hot, el término de intersección (intercepto) representa la media global de la variable objetivo: 'Renta', y cada uno de los coeficientes lineales representa que tanto difiere la renta media de esa ciudad respecto a la media global.

Ahora entrenemos el mismo modelo con dummy coding

Con codificación dummy, el coeficiente de sesgo (intercepto) representa el valor medio de la variable $y$ para la categoría de referencia, que en el ejemplo es la ciudad NYC. El coeficiente para la *i*-ésima característica es igual a la diferencia entre el valor medio de la respuesta para la i-ésima categoría y la media de la categoría de referencia.

### Dummy Coding


### Sum (or Deviation) Coding

Utilizando el ejemplo de prueba

La codificación de efectos (sum coding) es muy similar a la codificación dummy, pero da como resultado modelos de regresión lineal que son aún más simples de interpretar.

En el ejemplo, vemos que el término de intersección representa la media global del la variable respuesta, y los coeficientes individuales indican cuánto las medias de las categorías individuales difieren de la media global. (Esto se llama el efecto principal de la categoría o nivel, de ahí el nombre "codificación de efectos").

La codificación One-hot, encontró la misma intersección y coeficientes, pero en ese caso hay coeficientes lineales para cada ciudad. En la codificación de efecto, ningun feature representa la categoría de referencia, ***por lo que el efecto de la categoría de referencia debe calcularse por separado como la suma negativa de los coeficientes de todas las demás categorías.***

Restulados de la regresión lineal con One-hot encoding

Coefficients: $[ 166.66666667,   666.66666667,  -833.33333333]$

Intercept: $3333.3333333333335$

### Backward Difference Coding

Utilizando el ejemplo de prueba

La codificación de diferencia en reversa es útil para codificar variables ordinales.

En el ejemplo, vemos que el término de intersección representa la media global de la variable respuesta, y los coeficientes individuales indican cuánto las medias de las categorías individuales difieren de la media de la categoría inmediatamente anterior.

### Helmert Coding

Utilizando el ejemplo de prueba

#Supervised Encoders

## Target Encoder

## Leave-one-out Encoder

## M-estimate Encoder

## Weight of Evidence Encoder